In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Hemochromatosis/GSE50579'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression profiling of HCC"
!Series_summary	"61 human HCCs were analyzed for genome-wide gene expression. Samples were collected at two sites in Germany, Heidelberg (HD) and Hannover (N). The Heidelberg Collection include 40 independent HCC: 19 liver resections and 17 explant liver specimen (4 not determined); median age at surgery was 57 years (range, 16-78), and the male/female ratio was 3:1. All diagnoses were confirmed by histological reevaluation, and use of the samples was approved by the local ethics committee.  From 3 patients, two HCC nodules were included that previously showed different aCGH, indicating independent tumor development (tagged by _2). The underlying etiologies were HBV (n=8), HCV (n=9), alcohol (n=14), cryptogenic (n=11), genetic hemochromatosis (n=3), and Alpha-1 antitrypsin deficiency (n=2). The Hannover Collection include 21 HCC (HCC-Grad: G1-G3). Technical replicates were tagged with _B or _C"
!Series_overall_design	"

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if the dataset contains gene expression data
is_gene_available = True  # Based on the Series summary indicating genome-wide gene expression

# Variable Availability
sample_characteristics = {0: ['tumor stage: n.d.', 'tumor stage: HCC-Grad:G1', 'tumor stage: HCC-Grad:G2', 'tumor stage: HCC-Grad:G2/3', 'tumor stage: HCC-Grad:G3'], 1: ['etiology: alcohol abuse', 'etiology: cryptogenic', 'etiology: HCV', 'etiology: HBV', 'etiology: genetic hemochromatosis', 'etiology: n.d.', 'etiology: Alpha-1 antitrypsin deficiency'], 2: ['sample type: hepatocellular carcinoma (HCC)', 'sample type: cell line control', 'sample type: Normal liver tissue control'], 3: ['gender: male', 'gender: female', 'gender: n.d.'], 4: ['material: Tx', 'material: n.d.', 'material: Rx', 'material: Rx (Netz)'], 5: ['age (yrs): 57', 'age (yrs): 47', 'age (yrs): 54', 'age (yrs): 78', 'age (yrs): 35', 'age (yrs): 50', 'age (yrs): 38', 'age (yrs): 73', 'age (yrs): 55', 'age (yrs): 74', 'age (yrs): 58', 'age (yrs): 61', 'age (yrs): 53', 'age (yrs): 65', 'age (yrs): 69', 'age (yrs): 72', 'age (yrs): 48', 'age (yrs): 51', 'age (yrs): 63', 'age (yrs): 16', 'age (yrs): 68', 'age (yrs): 40', 'age (yrs): n.d.', 'age (yrs): 70', 'age (yrs): 52', 'age (yrs): 46', 'age (yrs): 66', 'age (yrs): 45', 'age (yrs): 67'], 6: ['tumor size [cm]: 2.0', 'tumor size [cm]: 3.0', 'tumor size [cm]: 3.5', 'tumor size [cm]: x', 'tumor size [cm]: 8.0', 'tumor size [cm]: 4.0', 'tumor size [cm]: 6.0', 'tumor size [cm]: 29.0', 'tumor size [cm]: 13.0', 'tumor size [cm]: 2.8', 'tumor size [cm]: 1.8', 'tumor size [cm]: 12.0', 'tumor size [cm]: 5.0', 'tumor size [cm]: 2.3', 'tumor size [cm]: 9.0', 'tumor size [cm]: 2.5', 'tumor size [cm]: 6.5', 'tumor size [cm]: 11.0', 'tumor size [cm]: 11.5', 'tumor size [cm]: 4.5', 'tumor size [cm]: 7.0', 'tumor size [cm]: 16.0', 'tumor size [cm]: n.d.', 'tumor size [cm]: 15.0', 'tumor size [cm]: 12', 'tumor size [cm]: 5.1']}

# Identification of keys for each variable
# Hemochromatosis is part of the etiology variable
if 1 in sample_characteristics and 'etiology: genetic hemochromatosis' in sample_characteristics[1]:
    trait_row = 1
else:
    trait_row = None

# Age is explicitly available
if 5 in sample_characteristics and len(set(sample_characteristics[5])) > 1:
    age_row = 5
else:
    age_row = None

# Gender is explicitly available
if 3 in sample_characteristics and len(set(sample_characteristics[3])) > 1:
    gender_row = 3
else:
    gender_row = None

# Data Type Conversion Functions
def convert_trait(value):
    if "genetic hemochromatosis" in value:
        return 1
    elif "etiology:" in value:
        return 0
    return None

def convert_age(value):
    try:
        return float(value.split(": ")[1])
    except ValueError:
        return None

def convert_gender(value):
    gender = value.split(": ")[1].lower()
    if gender == 'male':
        return 1
    elif gender == 'female':
        return 0
    return None

# Save Metadata
save_cohort_info('GSE50579', './preprocessed/Hemochromatosis/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Hemochromatosis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Hemochromatosis/trait_data/GSE50579.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1223814': [0.0, 57.0, 1.0], 'GSM1223815': [0.0, 57.0, 1.0], 'GSM1223816': [0.0, 57.0, 1.0], 'GSM1223817': [0.0, 47.0, 1.0], 'GSM1223818': [0.0, 54.0, 1.0], 'GSM1223819': [0.0, 78.0, 1.0], 'GSM1223820': [0.0, 35.0, 1.0], 'GSM1223821': [0.0, 50.0, 0.0], 'GSM1223822': [0.0, 38.0, 0.0], 'GSM1223823': [0.0, 73.0, 0.0], 'GSM1223824': [0.0, 55.0, 1.0], 'GSM1223825': [0.0, 74.0, 0.0], 'GSM1223826': [0.0, 58.0, 1.0], 'GSM1223827': [0.0, 58.0, 1.0], 'GSM1223828': [0.0, 61.0, 0.0], 'GSM1223829': [0.0, 53.0, 1.0], 'GSM1223830': [1.0, 65.0, 1.0], 'GSM1223831': [0.0, 69.0, 0.0], 'GSM1223832': [0.0, 72.0, 1.0], 'GSM1223833': [0.0, 54.0, 1.0], 'GSM1223834': [0.0, 54.0, 1.0], 'GSM1223835': [0.0, 38.0, 0.0], 'GSM1223836': [0.0, 48.0, 1.0], 'GSM1223837': [0.0, 73.0, 1.0], 'GSM1223838': [0.0, 51.0, 1.0], 'GSM1223839': [0.0, 63.0, 0.0], 'GSM1223840': [0.0, 16.0, 1.0], 'GSM1223841': [0.0, 78.0, 1.0], 'GSM1223842': [0.0, 69.0, 1.0], 'GSM1223843': [0.0, 68.0, 1.0], 'GSM1223844': [0.0, 40.0, 1.0], 'GSM122

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107',
       '(+)E1A_r60_a135', '(+)E1A_r60_a20', '(+)E1A_r60_a22', '(+)E1A_r60_a97',
       '(+)E1A_r60_n11', '(+)E1A_r60_n9', '3xSLv1', 'A_19_P00315452',
       'A_19_P00315459', 'A_19_P00315469', 'A_19_P00315473', 'A_19_P00315482',
       'A_19_P00315490', 'A_19_P00315492', 'A_19_P00315493', 'A_19_P00315496'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107', '(+)E1A_r60_a135'], 'SPOT_ID': ['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107', '(+)E1A_r60_a135'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'REFSEQ': [nan, nan, nan, nan, nan], 'GB_ACC': [nan, nan, nan, nan, nan], 'GENE': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'UNIGENE_ID': [nan, nan, nan, nan, nan], 'ENSEMBL_ID': [nan, nan, nan, nan, nan], 'TIGR_ID': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan], 'CYTOBAND': [nan, nan, nan, nan, nan], 'DESCRIPTION': [nan, nan, nan, nan, nan], 'GO_ID': [nan, nan, nan, nan, nan], 'SEQUENCE': [nan, nan, nan, nan, nan]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Set the identifier_key and gene_symbol_key based on the printed dictionary from STEP5
identifier_key = 'ID'
gene_symbol_key = 'GENE_SYMBOL'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping = get_gene_mapping(gene_annotation, prob_col=identifier_key, gene_col=gene_symbol_key)

# 3. Apply the mapping to get the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Hemochromatosis/gene_data/GSE50579.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Hemochromatosis')

# 4. Save the cohort information.
save_cohort_info('GSE50579', './preprocessed/Hemochromatosis/cohort_info.json', True, True, trait_biased, merged_data)

# 5. If the trait is not severely biased, save the merged data to a csv file.
if not trait_biased:
    csv_path = './preprocessed/Hemochromatosis/GSE50579.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Hemochromatosis', the least common label is '1.0' with 2 occurrences. This represents 4.26% of the dataset.
The distribution of the feature 'Hemochromatosis' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 51.5
  50% (Median): 55.0
  75%: 67.5
Min: 16.0
Max: 78.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 13 occurrences. This represents 27.66% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

A new JSON file was created at: ./preprocessed/Hemochromatosis/cohort_info.json
